In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
from keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler,OneHotEncoder
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical

In [2]:
df= pd.read_csv('clean_data.csv')
df

,Protocol,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,...,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,112641719,3,0,0,0,0,0,0.000000,0.000000,...,0,0.0,0.0,0,0,56320859.5,139.300036,56320958,56320761,Benign
1,0,112641466,3,0,0,0,0,0,0.000000,0.000000,...,0,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign
2,0,112638623,3,0,0,0,0,0,0.000000,0.000000,...,0,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign
3,6,6453966,15,10,1239,2273,744,0,82.600000,196.741237,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
4,6,8804066,14,11,1143,2209,744,0,81.642857,203.745545,...,32,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798299,6,19276110,5,10,286,738,103,0,57.200000,52.485236,...,20,9667037.0,0.0,9667037,9667037,9609073.0,0.000000,9609073,9609073,Benign
2798300,6,85992425,2,0,0,0,0,0,0.000000,0.000000,...,20,0.0,0.0,0,0,85992425.0,0.000000,85992425,85992425,Benign
2798301,17,79134,1,1,41,98,41,41,41.000000,0.000000,...,8,0.0,0.0,0,0,0.0,0.000000,0,0,Benign
2798302,6,10053785,11,12,1321,1659,677,0,120.090909,197.116440,...,20,0.0,0.0,0,0,0.0,0.000000,0,0,Benign


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2798304 entries, 0 to 2798303
Data columns (total 70 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Protocol           int64  
 1   Flow Duration      int64  
 2   Tot Fwd Pkts       int64  
 3   Tot Bwd Pkts       int64  
 4   TotLen Fwd Pkts    int64  
 5   TotLen Bwd Pkts    int64  
 6   Fwd Pkt Len Max    int64  
 7   Fwd Pkt Len Min    int64  
 8   Fwd Pkt Len Mean   float64
 9   Fwd Pkt Len Std    float64
 10  Bwd Pkt Len Max    int64  
 11  Bwd Pkt Len Min    int64  
 12  Bwd Pkt Len Mean   float64
 13  Bwd Pkt Len Std    float64
 14  Flow Byts/s        float64
 15  Flow Pkts/s        float64
 16  Flow IAT Mean      float64
 17  Flow IAT Std       float64
 18  Flow IAT Max       int64  
 19  Flow IAT Min       int64  
 20  Fwd IAT Tot        int64  
 21  Fwd IAT Mean       float64
 22  Fwd IAT Std        float64
 23  Fwd IAT Max        int64  
 24  Fwd IAT Min        int64  
 25  Bwd IAT Tot       

In [4]:
df.columns

Index(['Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Subflow Fwd Pkts'

In [5]:
df['Label'].unique()

array(['Benign', 'FTP-BruteForce', 'SSH-Bruteforce', 'Infilteration',
       'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection'],
      dtype=object)

In [6]:
metadata=[ 'Protocol', 'Flow Duration', 'Tot Fwd Pkts', 'Tot Bwd Pkts',
       'TotLen Fwd Pkts', 'TotLen Bwd Pkts', 'Fwd Pkt Len Max',
       'Fwd Pkt Len Min', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Max', 'Bwd Pkt Len Min', 'Bwd Pkt Len Mean',
       'Bwd Pkt Len Std', 'Flow Byts/s', 'Flow Pkts/s', 'Flow IAT Mean',
       'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Tot',
       'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
       'Bwd IAT Tot', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
       'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Len',
       'Bwd Header Len', 'Fwd Pkts/s', 'Bwd Pkts/s', 'Pkt Len Min',
       'Pkt Len Max', 'Pkt Len Mean', 'Pkt Len Std', 'Pkt Len Var',
       'FIN Flag Cnt', 'SYN Flag Cnt', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'URG Flag Cnt', 'CWE Flag Count', 'ECE Flag Cnt',
       'Down/Up Ratio', 'Pkt Size Avg', 'Fwd Seg Size Avg', 'Bwd Seg Size Avg',
       'Subflow Fwd Pkts', 'Subflow Fwd Byts', 'Subflow Bwd Pkts',
       'Subflow Bwd Byts', 'Init Fwd Win Byts', 'Init Bwd Win Byts',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min'] #before feature extraction

In [7]:
## After feature extraction

features= ['Flow Duration', 'Tot Bwd Pkts', 'TotLen Fwd Pkts', 'Flow Byts/s',
       'Flow Pkts/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Max',
       'Bwd IAT Std', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s',
       'Bwd Pkts/s', 'Pkt Len Max', 'Pkt Len Std', 'Pkt Size Avg',
       'Subflow Fwd Byts', 'Subflow Bwd Pkts', 'Subflow Bwd Byts',
       'Init Fwd Win Byts', 'Fwd Act Data Pkts', 'Fwd Seg Size Min']

In [8]:
categories= ['Benign', 'FTP-BruteForce', 'SSH-Bruteforce', 'Infilteration',
       'Brute Force -Web', 'Brute Force -XSS', 'SQL Injection']

In [9]:
X = df[features]
X[features] = X[features].apply(pd.to_numeric, errors='coerce', axis=1)
X = X.fillna(0)

C:\Users\aryan\anaconda3\lib\site-packages\pandas\core\frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [10]:
X

,Flow Duration,Tot Bwd Pkts,TotLen Fwd Pkts,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,...,Bwd Pkts/s,Pkt Len Max,Pkt Len Std,Pkt Size Avg,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min
0,112641719.0,0.0,0.0,0.000000,0.026633,5.632086e+07,1.393000e+02,56320958.0,56320761.0,112641719.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,-1.0,0.0,0.0
1,112641466.0,0.0,0.0,0.000000,0.026633,5.632073e+07,1.145513e+02,56320814.0,56320652.0,112641466.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,-1.0,0.0,0.0
2,112638623.0,0.0,0.0,0.000000,0.026634,5.631931e+07,3.019346e+02,56319525.0,56319098.0,112638623.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,-1.0,0.0,0.0
3,6453966.0,10.0,1239.0,544.161528,3.873587,2.689152e+05,2.474438e+05,673900.0,22.0,6453966.0,...,1.549435,976.0,277.834760,140.480000,1239.0,10.0,2273.0,65535.0,6.0,32.0
4,8804066.0,11.0,1143.0,380.733175,2.839597,3.668361e+05,5.113566e+05,1928102.0,21.0,8804066.0,...,1.249423,976.0,279.763032,134.080000,1143.0,11.0,2209.0,5808.0,6.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2798299,19276110.0,10.0,286.0,53.122751,0.778165,1.376865e+06,2.727253e+06,9609073.0,38.0,322786.0,...,0.518777,409.0,99.087167,68.266667,286.0,10.0,738.0,8192.0,3.0,20.0
2798300,85992425.0,0.0,0.0,0.000000,0.023258,8.599242e+07,0.000000e+00,85992425.0,85992425.0,85992425.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,62569.0,0.0,20.0
2798301,79134.0,1.0,41.0,1756.514267,25.273587,7.913400e+04,0.000000e+00,79134.0,79134.0,0.0,...,12.636793,98.0,32.908965,90.000000,41.0,1.0,98.0,-1.0,0.0,8.0
2798302,10053785.0,12.0,1321.0,296.405782,2.287696,4.569902e+05,8.706840e+05,3013611.0,4.0,10053785.0,...,1.193580,1173.0,267.679508,129.565217,1321.0,12.0,1659.0,8192.0,7.0,20.0


In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2798304 entries, 0 to 2798303
Data columns (total 26 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Flow Duration      float64
 1   Tot Bwd Pkts       float64
 2   TotLen Fwd Pkts    float64
 3   Flow Byts/s        float64
 4   Flow Pkts/s        float64
 5   Flow IAT Mean      float64
 6   Flow IAT Std       float64
 7   Flow IAT Max       float64
 8   Flow IAT Min       float64
 9   Fwd IAT Tot        float64
 10  Fwd IAT Mean       float64
 11  Fwd IAT Max        float64
 12  Bwd IAT Std        float64
 13  Fwd Header Len     float64
 14  Bwd Header Len     float64
 15  Fwd Pkts/s         float64
 16  Bwd Pkts/s         float64
 17  Pkt Len Max        float64
 18  Pkt Len Std        float64
 19  Pkt Size Avg       float64
 20  Subflow Fwd Byts   float64
 21  Subflow Bwd Pkts   float64
 22  Subflow Bwd Byts   float64
 23  Init Fwd Win Byts  float64
 24  Fwd Act Data Pkts  float64
 25  Fwd Seg Size Min  

In [12]:
#labels = df['Label']
#encoder = LabelEncoder()
#y= encoder.fit_transform(labels)
#y = to_categorical(y, num_classes=7)

In [13]:
#labels = df['Label']
#Create an instance of OneHotEncoder
#encoder = OneHotEncoder()

#Fit and transform the 'Label' column
#y = encoder.fit_transform(labels.values.reshape(-1, 1)).toarray()

In [14]:
y= df['Label']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(2238643, 26) (2238643,)
(559661, 26) (559661,)


In [15]:
#selecting 1% of random rows for better running time

X_train = X_train.sample(frac=0.1, replace=True, random_state=1)
y_train = y_train.sample(frac=0.1, replace=True, random_state=1)
X_test = X_test.sample(frac=0.1, replace=True, random_state=1)
y_test = y_test.sample(frac=0.1, replace=True, random_state=1)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(223864, 26) (223864,)
(55966, 26) (55966,)


In [16]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_train.values)
X_train = pd.DataFrame(x_scaled,columns=features)
x_scaled_test = min_max_scaler.fit_transform(X_test.values)
X_test = pd.DataFrame(x_scaled_test,columns=features)

In [21]:
# Create an instance of OneHotEncoder
labels = df['Label'] 
encoder = OneHotEncoder()

# Fit and transform the
y_train = encoder.fit_transform(y_train.values.reshape(-1, 1)).toarray()
y_test = encoder.fit_transform(y_test.values.reshape(-1, 1)).toarray()

In [22]:
y_test

array([[0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [23]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [24]:
X_train.values

array([[4.37500029e-06, 0.00000000e+00, 1.02686410e-05, ...,
        3.89099121e-03, 1.47863374e-04, 4.54545455e-01],
       [9.73540065e-03, 2.89077018e-04, 1.50428922e-04, ...,
        1.25015259e-01, 7.39316871e-04, 4.54545455e-01],
       [1.57566094e-02, 3.30373735e-04, 1.53096102e-04, ...,
        1.25015259e-01, 7.39316871e-04, 4.54545455e-01],
       ...,
       [1.41476176e-02, 4.95560603e-04, 2.68985050e-04, ...,
        1.25015259e-01, 8.87180245e-04, 4.54545455e-01],
       [7.18961248e-01, 0.00000000e+00, 0.00000000e+00, ...,
        9.54589844e-01, 0.00000000e+00, 4.54545455e-01],
       [2.44716266e-02, 2.89077018e-04, 1.50428922e-04, ...,
        1.25015259e-01, 7.39316871e-04, 4.54545455e-01]])

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(100000, 64, embeddings_regularizer='l2'), # since it doesn't consider "words," the embedding doesn't really matter
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64, activation='relu', use_bias=True, bias_regularizer='l2'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7, activation='sigmoid')
    # tf.keras.layers.Dense(1, activation='softmax') # loss too big
])

In [26]:
#labels = df['Label']
#encoder = LabelEncoder()
#y_train= encoder.fit_transform(y_train)
#y_train = to_categorical(y_train, num_classes=7)
#y_test = encoder.fit_transform(y_test)
#y_test = to_categorical(y_test, num_classes=7)

In [27]:
X_train.shape, y_train.shape

((223864, 26), (223864, 7))

In [1]:
model.compile(loss='categorical_crossentropy',
              # optimizer='sgd', # almost same
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
history = model.fit(X_train.values, y_train, epoch=3)

NameError: name 'model' is not defined